# IEEE Fraud Detect: part 03:
## Trying out Machine learning models

In [1]:
import os
import gc
gc.enable()
import time

import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore")

In [5]:
# Loading the normalized reduced dataset for training
train_data = pd.read_parquet(os.getcwd() + '\\pca_num_train.gzip')
target = pd.read_parquet(os.getcwd() + '\\target_var.gzip')

In [ ]:
# Only load for prediction or later for manipulaiton:
# test_data = pd.read_parquet(os.getcwd() + '\\Data\\test.gzip')

In [6]:
target

,isFraud
0,0
1,0
2,0
3,0
4,0
...,...
590535,0
590536,0
590537,0
590538,0
